# Working with metadata from python

An important aspect of OMERO - maybe *the* most important aspect of OMERO - is working with metadata. OMERO provides two essential ways of enriching your  data with metadata: Tag annotations and key-value pairs. In this notebook you will learn some essential ways of working with these  from python. Topics covered in this notebook include:

- How to add key-value pairs to an image
- How to add tags to an image
- How to set the pixel size of an image
- How to set the channel name of an image

Recommended previous read: [How to upload and download data from OMERO](omero.io.read_and_write)

In [5]:
import ezomero
import omero
import numpy as np

In [3]:
host = 'omero-int.biotec.tu-dresden.de'
user = 'johamuel'  # replace this with your username
secure = True
port = 4064
group = 'default'

conn = ezomero.connect(host=host, user=user, secure=secure, port=port, group=group)

For the sake of this tutorial, we shall quickly upload some random data:

In [9]:
image = np.random.random((1, 1, 32, 32, 32))
image_id = ezomero.post_image(conn, image=image, dim_order='tczyx', image_name='random_image')
print('Image ID: ', image_id)

Image ID:  8755


## Key-value pairs

A key thing (pun intended) we would like to do, is to add some [key-value pairs](https://docs.openmicroscopy.org/omero/5.1.4/developers/Model/KeyValuePairs.html) to the image. These give us the opportunity to provide metadata to our image data in a very flexible way. On the Python side, a key-value pair is simply a dictionary, which we can create like this:

In [10]:
key_value_pairs = {
    'Owner': 'Johannes',
    'Date': '20230620'
}

Suppose we want to add some kv-pairs to the above-uploaded image, we can do it like this. The arguments are relatively straightforward, except for the `ns` argument, which is short for an [omero namespace](https://docs.openmicroscopy.org/omero/5.6.4/developers/Model/StructuredAnnotations.html#names-and-namespaces). The concept of the `namespace` ensure a few things:

- **Uniqueness**: Namespaces help avoid conflicts by making sure that annotations from different sources or with different purposes don't get mixed up or overwrite each other.
- **Context**: They provide a context for the data, making it clear what the data represents and how it should be interpreted.
- **Scalability**: As your OMERO server grows with more data and more users, namespaces help keep annotations organized and manageable.

Using a namespace has a very practical effect: 

In [14]:
ezomero.post_map_annotation(conn, 'Image', 8755, key_value_pairs, ns="openmicroscopy.org/omero/client/mapAnnotation")

object group 3


4030

The next thing we may want to do is to add a tag to our data. Again, we can use ezomero for this job. To do this, we first check whether a tag of a given name already exists in the space of available tags. If so, we can retrieve its id and use it to tag our data. If not, we can create a new tag and use its id to tag our data.

In [ ]:
from omero.gateway import TagAnnotationWrapper, TagAnnotationI
from omero.rtypes import rstring

In [ ]:
tag_name = "SampleData"  # Replace with your desired tag name
tags = conn.getObjects("TagAnnotation", attributes={"textValue": tag_name})

tag_id = None
for tag in tags:
    tag_id = tag.getId()
    break

# If the tag does not exist, create it
# If the tag does not exist, create it
if tag is None:
    tag = TagAnnotationWrapper(conn)
    tag.setValue(rstring(tag_name))
    tag.setDescription(rstring("A tag for sample data"))
    tag.save()


<TagAnnotationWrapper id=4024>

In [ ]:

image_object = conn.getObject("Image", image_id)
image_object.linkAnnotation(tag)